In [11]:
from jupyter_plotly_dash import JupyterDash

import base64 #necessary import for image
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


# CRUD Python module file name and class name
from projectone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# username and password and CRUD Python module name

username = "aacuser"
password = "aacUserPass"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # company logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Travis Garlick, SNHU CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Div(        
    #code for interactive filtering Radio buttons
        dcc.RadioItems(
            ['Water Rescue', 'Wilderness Rescue', 'Tracking', 'Reset'],
            'Reset',
            #inline=True
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #features for interactive data table to make it user-friendly for client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that chart and geolocation are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        #if button is Water Rescue(
        #filter for breed = labrador retriever mix, Chesapeake Bay Retriever, Newfoundland
        #filter for sex = intact female
        #filter for age = 26 weeks to 156 weeks
        #)
        
        #if button is Wilderness Rescue(
        #filter for breed = German Shepherd, Alaskan Malamute, Old English Sheepdog, 
            #Siberian Husky, Rottweiler
        #filter for sex = intact male
        #filter for age = 26 weeks to 156 weeks
        #)
        
        #if button is Tracking(
        #filter for breed = Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, 
            #Rottweiler
        #filter for sex = intact male
        #filter for age = 20 weeks to 300 weeks
        #)        

        #if button is Reset(
        #no filter
        #)
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
#def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #df = px.data.animals()
    #percentages = animals.find({'breed'=breed}).count / animals.count_document({})
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, values='percentages', names='breed')
    #    )    
    #]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app